# Piper TTS Model Export

This notebook exports your trained Piper TTS model to ONNX format for inference.

## Prerequisites
- Completed at least 1 epoch of training
- Checkpoint file exists in `training_dir/lightning_logs/`

### 1. Setup Environment

In [1]:
import os
import sys
from pathlib import Path
import glob

# Set paths
piper_src_path = os.path.abspath("../piper_repo/src/python")
training_dir = os.path.abspath("training_dir")
export_dir = os.path.abspath("exported_models")

# Create export directory
os.makedirs(export_dir, exist_ok=True)

print(f"Piper Source: {piper_src_path}")
print(f"Training Dir: {training_dir}")
print(f"Export Dir: {export_dir}")

Piper Source: /Users/rutwik/piper-model-training/piper_repo/src/python
Training Dir: /Users/rutwik/piper-model-training/scripts/training_dir
Export Dir: /Users/rutwik/piper-model-training/scripts/exported_models


### 2. Find Latest Checkpoint

In [2]:
# Find all checkpoint files
checkpoint_pattern = os.path.join(training_dir, "lightning_logs/*/checkpoints/*.ckpt")
checkpoints = glob.glob(checkpoint_pattern)

if not checkpoints:
    print("❌ No checkpoints found!")
    print(f"Searched in: {checkpoint_pattern}")
    print("\nMake sure you've completed at least 1 epoch of training.")
else:
    # Sort by modification time, get latest
    latest_checkpoint = max(checkpoints, key=os.path.getmtime)
    checkpoint_name = Path(latest_checkpoint).stem
    
    print(f"✅ Found {len(checkpoints)} checkpoint(s)")
    print(f"\nLatest checkpoint: {checkpoint_name}")
    print(f"Full path: {latest_checkpoint}")

❌ No checkpoints found!
Searched in: /Users/rutwik/piper-model-training/scripts/training_dir/lightning_logs/*/checkpoints/*.ckpt

Make sure you've completed at least 1 epoch of training.


### 3. Export to ONNX Format

This exports the model using the `piper_train.export_onnx` module.

In [ ]:
if not checkpoints:
    print("⚠️ Skipping export - no checkpoint available")
else:
    # Define output paths
    output_model = os.path.join(export_dir, f"{checkpoint_name}.onnx")
    output_config = f"{output_model}.json"
    
    print(f"Exporting model to: {output_model}")
    print("This may take a few minutes...\n")
    
    # Run export command
    !PYTHONPATH="{piper_src_path}" "{sys.executable}" -m piper_train.export_onnx \
        "{latest_checkpoint}" \
        "{output_model}"
    
    # Copy config file
    config_source = os.path.join(training_dir, "config.json")
    if os.path.exists(config_source):
        import shutil
        shutil.copy(config_source, output_config)
        print(f"\n✅ Export complete!")
        print(f"\nModel files:")
        print(f"  - {output_model}")
        print(f"  - {output_config}")
    else:
        print(f"\n⚠️ Warning: Could not find config.json at {config_source}")

### 4. Test Exported Model (Optional)

If you have Piper CLI installed, you can test the exported model.

In [ ]:
# Test if piper is installed
import subprocess

try:
    subprocess.run(["piper", "--version"], check=True, capture_output=True)
    piper_available = True
    print("✅ Piper CLI is available")
except (subprocess.CalledProcessError, FileNotFoundError):
    piper_available = False
    print("ℹ️ Piper CLI not found - skipping test")
    print("Install with: pip install piper-tts")

In [ ]:
if piper_available and checkpoints:
    test_text = "Hello world, this is a test of the exported Piper model."
    test_output = os.path.join(export_dir, "test.wav")
    
    print(f"Testing with text: '{test_text}'")
    !echo "{test_text}" | piper -m "{output_model}" --output_file "{test_output}"
    
    if os.path.exists(test_output):
        print(f"\n✅ Test audio generated: {test_output}")
        # Display audio player if in Jupyter
        try:
            from IPython.display import Audio, display
            display(Audio(test_output))
        except:
            pass
    else:
        print("\n❌ Test failed - no audio file generated")

### 5. Model Information

In [ ]:
if checkpoints:
    import json
    
    # Display model info
    print("="*60)
    print("EXPORTED MODEL INFORMATION")
    print("="*60)
    print(f"\nCheckpoint: {checkpoint_name}")
    print(f"Model file: {output_model}")
    print(f"Config file: {output_config}")
    
    # Get file sizes
    if os.path.exists(output_model):
        model_size_mb = os.path.getsize(output_model) / (1024 * 1024)
        print(f"\nModel size: {model_size_mb:.2f} MB")
    
    # Show config
    if os.path.exists(output_config):
        with open(output_config, 'r') as f:
            config = json.load(f)
        print(f"\nConfig details:")
        print(f"  Sample rate: {config.get('audio', {}).get('sample_rate', 'N/A')} Hz")
        print(f"  Num speakers: {config.get('num_speakers', 'N/A')}")
        print(f"  Num symbols: {config.get('num_symbols', 'N/A')}")
    
    print("\n" + "="*60)
    print("\n✅ Export complete! You can now use this model with Piper.")
    print(f"\nUsage example:")
    print(f'  echo "Your text here" | piper -m {output_model} --output_file output.wav')